In [ ]:
!pip install understat
from understat import Understat

INFO: pip is looking at multiple versions of pytest-asyncio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.0/244.0 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 8.3.5
    Uninstalling pytest-8.3.5:
      Successfully uninstalled pytest-8.3.5
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.4.1
    Uninstalling charset-normalizer-3.4.1:
      Successfully uninstalled charset-normalizer-3.4.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.4
    Uninstalling beautifulsoup4-4.13.4:
      Successfully uninstalled beautifulsoup

In [ ]:
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta
import nest_asyncio
import json

path_dict = {
    'EPL': 'EPL',
    'La Liga': 'LaLiga',
    'Bundesliga': 'Bundesliga',
    'Serie A': 'SerieA',
    'Ligue 1': 'Ligue1',
    }

leagues = ['EPL', 'La Liga', 'Bundesliga', 'Serie A', 'Ligue 1']

# Data

## Generate Data

In [ ]:
# Aplicando a correção no loop
nest_asyncio.apply()


async def get_season_results(session, league, season):
    understat = Understat(session)
    return await understat.get_league_results(league, season)

async def get_table_until_date(session, league, season, end_date, h_a="overall"):
    understat = Understat(session)
    start_date = f"{season}-01-01"
    return await understat.get_league_table(
        league, season, start_date=start_date, end_date=end_date, h_a=h_a
    )

async def collect_data(league, seasons):
    async with aiohttp.ClientSession() as session:
        league = league
        seasons = seasons
        columns = [
            'match_id', 'date', 'home_team_id', 'home_team_name',
            'away_team_id', 'away_team_name', 'home_goals', 'away_goals'
        ] + [f'{stat}_{side}' for stat in [
            'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
            'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'] for side in ['h', 'a']]

        data = pd.DataFrame(columns=columns)

        if seasons[0] == 2024:
          data = pd.read_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results_2024.csv')

        for season in seasons:
            fixtures = await get_season_results(session, league, season)
            count = 0
            await asyncio.sleep(1)
            for fixture in fixtures:
                match_id = fixture['id']
                if int(match_id) in data['match_id'].unique():
                  pass
                else:
                  match_date = datetime.strptime(fixture['datetime'], "%Y-%m-%d %H:%M:%S")
                  end_date = (match_date - timedelta(days=1)).strftime("%Y-%m-%d")

                  table = await get_table_until_date(session, league, str(season), end_date)
                  table_df = pd.DataFrame(table[1:], columns=table[0])

                  home_team = fixture['h']
                  away_team = fixture['a']

                  home_stats = table_df.loc[table_df['Team'] == home_team['title']].squeeze()
                  away_stats = table_df.loc[table_df['Team'] == away_team['title']].squeeze()

                  row = {
                      'match_id': match_id,
                      'date': match_date,
                      'home_team_id': home_team['id'],
                      'home_team_name': home_team['title'],
                      'away_team_id': away_team['id'],
                      'away_team_name': away_team['title'],
                      'home_goals': fixture['goals']['h'],
                      'away_goals': fixture['goals']['a'],
                  }

                  for stat in [
                      'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                      'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
                  ]:
                      row[f'{stat}_h'] = home_stats[stat]
                      row[f'{stat}_a'] = away_stats[stat]

                  data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)
                  # count += 1
                  # if count >= 20:
                  #   break

        if seasons[0] == 2024:
          data.to_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results_2024.csv', index=False)
        else:
          data.to_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results.csv', index=False)

async def collect_data_(league, seasons):
    async with aiohttp.ClientSession() as session:
        league = league
        seasons = seasons
        # seasons = range(2024, 2025)
        columns = [
            'match_id', 'date', 'home_team_id', 'home_team_name',
            'away_team_id', 'away_team_name', 'home_goals', 'away_goals'
        ] + [
            f'{stat}_{side}' for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ] for side in ['h', 'a']
        ] + [
            f'{stat}_h_at_h' for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ]
        ] + [
            f'{stat}_a_at_a' for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ]
        ]

        data = pd.DataFrame(columns=columns)

        if seasons[0] == 2024:
          data = pd.read_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results_2024_.csv')

        for season in seasons:
            fixtures = await get_season_results(session, league, season)
            count = 0
            await asyncio.sleep(1)
            for fixture in fixtures:
                match_id = fixture['id']
                if int(match_id) in data['match_id'].unique():
                  pass
                else:
                  match_date = datetime.strptime(fixture['datetime'], "%Y-%m-%d %H:%M:%S")
                  end_date = (match_date - timedelta(days=1)).strftime("%Y-%m-%d")

                  # Tabela geral
                  general_table = await get_table_until_date(session, league, str(season), end_date)
                  general_df = pd.DataFrame(general_table[1:], columns=general_table[0])

                  # Tabela jogando em casa
                  home_table = await get_table_until_date(session, league, str(season), end_date, h_a="home")
                  home_df = pd.DataFrame(home_table[1:], columns=home_table[0])

                  # Tabela jogando fora
                  away_table = await get_table_until_date(session, league, str(season), end_date, h_a="away")
                  away_df = pd.DataFrame(away_table[1:], columns=away_table[0])

                  home_team = fixture['h']
                  away_team = fixture['a']

                  # Estatísticas gerais
                  home_stats = general_df.loc[general_df['Team'] == home_team['title']].squeeze()
                  away_stats = general_df.loc[general_df['Team'] == away_team['title']].squeeze()

                  # Estatísticas jogando em casa e fora
                  home_stats_at_home = home_df.loc[home_df['Team'] == home_team['title']].squeeze()
                  away_stats_at_away = away_df.loc[away_df['Team'] == away_team['title']].squeeze()

                  row = {
                      'match_id': match_id,
                      'date': match_date,
                      'home_team_id': home_team['id'],
                      'home_team_name': home_team['title'],
                      'away_team_id': away_team['id'],
                      'away_team_name': away_team['title'],
                      'home_goals': fixture['goals']['h'],
                      'away_goals': fixture['goals']['a'],
                  }

                  for stat in [
                      'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                      'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
                  ]:
                      row[f'{stat}_h'] = home_stats[stat]
                      row[f'{stat}_a'] = away_stats[stat]
                      row[f'{stat}_h_at_h'] = home_stats_at_home[stat]
                      row[f'{stat}_a_at_a'] = away_stats_at_away[stat]

                  data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)

        if seasons[0] == 2024:
          data.to_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results_2024_.csv', index=False)
        else:
          data.to_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results_.csv', index=False)


In [ ]:

for league in leagues:
  # await collect_data(league, range(2014, 2024))
  await collect_data(league, range(2024, 2025))
  # await collect_data_(league, range(2014, 2024))
  await collect_data_(league, range(2024, 2025))

## Generate Pred

In [ ]:
# Aplicando a correção no loop
nest_asyncio.apply()

async def get_season_fixtures(session, league, season):
    understat = Understat(session)

    # Data de hoje
    hoje = datetime.now()

    # Calcula a segunda-feira ao meio-dia
    first_day = (hoje - timedelta(days=hoje.weekday())).replace(hour=0, minute=0, second=0, microsecond=0)

    # Calcula o domingo ao meio-dia
    last_day = first_day + timedelta(days=7)

    start_date = first_day.strftime("%Y-%m-%d")
    end_date = last_day.strftime("%Y-%m-%d")

    fixtures = await understat.get_league_fixtures(league, season)

    if start_date and end_date:
        # Filtrando pelo intervalo de datas
        filtered_fixtures = [
            fixture for fixture in fixtures
            if start_date <= fixture['datetime'].split(' ')[0] <= end_date
        ]
    elif start_date:
        # Filtrando por uma data específica
        filtered_fixtures = [
            fixture for fixture in fixtures
            if fixture['datetime'].startswith(start_date)
        ]

    return filtered_fixtures

async def get_table_until_date(session, league, season, end_date, h_a="overall"):
    understat = Understat(session)
    start_date = f"{season}-01-01"
    return await understat.get_league_table(
        league, season, start_date=start_date, end_date=end_date, h_a=h_a
    )

async def generate_pred(league, season):
    async with aiohttp.ClientSession() as session:
        league = league
        season = season
        columns = [
            'match_id', 'date', 'home_team_id', 'home_team_name',
            'away_team_id', 'away_team_name', 'home_goals', 'away_goals'
        ] + [f'{stat}_{side}' for stat in [
            'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
            'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'] for side in ['h', 'a']]

        data = pd.DataFrame(columns=columns)

        fixtures = await get_season_fixtures(session, league, season)
        count = 0
        for fixture in fixtures:
            match_id = fixture['id']
            match_date = datetime.strptime(fixture['datetime'], "%Y-%m-%d %H:%M:%S")
            end_date = (match_date - timedelta(days=1)).strftime("%Y-%m-%d")

            table = await get_table_until_date(session, league, str(season), end_date)
            table_df = pd.DataFrame(table[1:], columns=table[0])

            home_team = fixture['h']
            away_team = fixture['a']

            home_stats = table_df.loc[table_df['Team'] == home_team['title']].squeeze()
            away_stats = table_df.loc[table_df['Team'] == away_team['title']].squeeze()

            row = {
                'match_id': match_id,
                'date': match_date,
                'home_team_id': home_team['id'],
                'home_team_name': home_team['title'],
                'away_team_id': away_team['id'],
                'away_team_name': away_team['title'],
                'home_goals': fixture['goals']['h'],
                'away_goals': fixture['goals']['a'],
            }

            for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ]:
                row[f'{stat}_h'] = home_stats[stat]
                row[f'{stat}_a'] = away_stats[stat]

            data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)
            # count += 1
            # if count >= 20:
            #   break

        data.to_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_pred.csv', index=False)
        # print(data.to_string())

async def generate_pred_(league, season):
    async with aiohttp.ClientSession() as session:
        league = league
        season = season
        columns = [
            'match_id', 'date', 'home_team_id', 'home_team_name',
            'away_team_id', 'away_team_name', 'home_goals', 'away_goals'
        ] + [
            f'{stat}_{side}' for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ] for side in ['h', 'a']
        ] + [
            f'{stat}_h_at_h' for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ]
        ] + [
            f'{stat}_a_at_a' for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ]
        ]

        data = pd.DataFrame(columns=columns)

        fixtures = await get_season_fixtures(session, league, season)
        count = 0
        for fixture in fixtures:
            match_id = fixture['id']
            match_date = datetime.strptime(fixture['datetime'], "%Y-%m-%d %H:%M:%S")
            end_date = (match_date - timedelta(days=1)).strftime("%Y-%m-%d")

            # Tabela geral
            general_table = await get_table_until_date(session, league, str(season), end_date)
            general_df = pd.DataFrame(general_table[1:], columns=general_table[0])

            # Tabela jogando em casa
            home_table = await get_table_until_date(session, league, str(season), end_date, h_a="home")
            home_df = pd.DataFrame(home_table[1:], columns=home_table[0])

            # Tabela jogando fora
            away_table = await get_table_until_date(session, league, str(season), end_date, h_a="away")
            away_df = pd.DataFrame(away_table[1:], columns=away_table[0])

            home_team = fixture['h']
            away_team = fixture['a']

            # Estatísticas gerais
            home_stats = general_df.loc[general_df['Team'] == home_team['title']].squeeze()
            away_stats = general_df.loc[general_df['Team'] == away_team['title']].squeeze()

            # Estatísticas jogando em casa e fora
            home_stats_at_home = home_df.loc[home_df['Team'] == home_team['title']].squeeze()
            away_stats_at_away = away_df.loc[away_df['Team'] == away_team['title']].squeeze()

            row = {
                'match_id': match_id,
                'date': match_date,
                'home_team_id': home_team['id'],
                'home_team_name': home_team['title'],
                'away_team_id': away_team['id'],
                'away_team_name': away_team['title'],
                'home_goals': fixture['goals']['h'],
                'away_goals': fixture['goals']['a'],
            }

            for stat in [
                'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA', 'NPxGA', 'NPxGD',
                'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS'
            ]:
                row[f'{stat}_h'] = home_stats[stat]
                row[f'{stat}_a'] = away_stats[stat]
                row[f'{stat}_h_at_h'] = home_stats_at_home[stat]
                row[f'{stat}_a_at_a'] = away_stats_at_away[stat]

            data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)
            # count += 1
            # if count >= 20:
            #   break

        data.to_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_pred_.csv', index=False)

In [ ]:
for league in leagues:
  await generate_pred(league, 2024)
  await generate_pred_(league, 2024)

<ipython-input-5-e1fb78d34291>:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)
<ipython-input-5-e1fb78d34291>:174: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame([row])], ignore_index=True)
<ipython-input-5-e1fb78d34291>:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns 

# ML

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

def train_models_rf(data, sample_weight):
  # Separar features e alvos
  X = data.drop(columns=['home_goals', 'away_goals'])
  y_home = data['home_goals']
  y_away = data['away_goals']

  # Criar sample_weight com base na data mais recente
  # sample_weight = np.linspace(1, 2, len(X))  # Pesos de 1 (antigo) a 2 (recente)

  # Treinar os modelos
  model_home = RandomForestRegressor(n_estimators=100, random_state=42)
  model_home.fit(X, y_home, sample_weight=sample_weight)

  model_away = RandomForestRegressor(n_estimators=100, random_state=42)
  model_away.fit(X, y_away, sample_weight=sample_weight)

  return model_home, model_away

def train_models_xgb(data, sample_weight):
  # Separar features e alvos
  X = data.drop(columns=['home_goals', 'away_goals'])
  y_home = data['home_goals']
  y_away = data['away_goals']

  # Criar sample_weight com base na data mais recente
  # sample_weight = np.linspace(1, 2, len(X))  # Pesos de 1 (antigo) a 2 (recente)

  # Treinar os modelos
  model_home = XGBRegressor(objective='reg:squarederror', random_state=42)
  model_home.fit(X, y_home, sample_weight=sample_weight)

  model_away = XGBRegressor(objective='reg:squarederror', random_state=42)
  model_away.fit(X, y_away, sample_weight=sample_weight)

  return model_home, model_away

In [ ]:
# Função para processar novos dados
def preprocess_new_data(new_data):
    features_to_drop = ['home_team_name', 'away_team_name', 'match_id', 'date', 'home_goals', 'away_goals']
    return new_data.drop(columns=features_to_drop)

# Função para fazer predições
def make_predictions(new_data_path, model_home, model_away):
    # Carregando novos dados
    new_data = pd.read_csv(new_data_path)

    if len(new_data) > 0:

      # Pré-processamento
      processed_data = preprocess_new_data(new_data)

      # Fazendo predições
      predictions_home = model_home.predict(processed_data)
      predictions_away = model_away.predict(processed_data)

      # Adicionando predições ao DataFrame
      new_data['home_goals'] = predictions_home
      new_data['away_goals'] = predictions_away

      # Arredondando os gols para inteiros
      new_data['home_goals_rounded'] = new_data['home_goals'].round().astype(int)
      new_data['away_goals_rounded'] = new_data['away_goals'].round().astype(int)

      # Arredondando os gols para inteiros
      new_data['home_goals_trunc'] = np.trunc(new_data['home_goals']).astype(int)
      new_data['away_goals_trunc'] = np.trunc(new_data['away_goals']).astype(int)

      return new_data
    else:
      return pd.DataFrame(columns=['date', 'home_team_name', 'away_team_name', 'home_goals', 'away_goals', 'home_goals_rounded', 'away_goals_rounded', 'home_goals_trunc', 'away_goals_trunc'])

# Exemplo de uso
# result = make_predictions('/content/drive/MyDrive/Datasets/FUT/Ligue1/football_pred.csv')
# print(result.to_string())

In [ ]:
# @title
from IPython.display import display_html

for league in leagues:
    consolidated_results = []  # Lista para armazenar os resultados de todas as iterações
    for data_f in ['', '_']:
        for sw in ['Linear', 'Log(x) + x']:
            # Carregar os dados
            df_pre = pd.read_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results{data_f}.csv')
            df_24 = pd.read_csv(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_results_2024{data_f}.csv')
            data = pd.concat([df_pre, df_24], ignore_index=True)

            team_mapping = pd.concat([
                data[['home_team_id', 'home_team_name']].rename(columns={'home_team_id': 'team_id', 'home_team_name': 'team_name'}),
                data[['away_team_id', 'away_team_name']].rename(columns={'away_team_id': 'team_id', 'away_team_name': 'team_name'})
            ]).drop_duplicates().set_index('team_id')['team_name'].to_dict()

            # Remover colunas irrelevantes
            data = data.drop(columns=['home_team_name', 'away_team_name', 'match_id', 'date'])

            if sw == 'Linear':
                # Criar sample_weight com base na data mais recente
                sample_weight = np.linspace(1, 2, len(data))  # Pesos de 1 (antigo) a 2 (recente)
                model_home_rf, model_away_rf = train_models_rf(data, sample_weight)
                model_home_xgb, model_away_xgb = train_models_xgb(data, sample_weight)

            elif sw == 'Log(x) + x':
                # Função ajustada para calcular os pesos com base em (log(x) + x) normalizada no intervalo [1, 2]
                sample_weights_ = np.linspace(1, 2, len(data))
                sample_weights_ = np.log(sample_weights_) + sample_weights_

                # Normalizando os pesos para o intervalo [1, 2]
                sample_weights_ = 1 + (sample_weights_ - sample_weights_.min()) / (sample_weights_.max() - sample_weights_.min())
                sample_weight = sample_weights_
                model_home_rf, model_away_rf = train_models_rf(data, sample_weight)
                model_home_xgb, model_away_xgb = train_models_xgb(data, sample_weight)

            # Fazer previsões para Random Forest e XGBoost
            result_rf = make_predictions(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_pred{data_f}.csv', model_home_rf, model_away_rf)
            result_xgb = make_predictions(f'/content/drive/MyDrive/Datasets/FUT/{path_dict[league]}/football_pred{data_f}.csv', model_home_xgb, model_away_xgb)

            # Adicionar os resultados à lista consolidada
            consolidated_results.append(result_rf[['home_goals', 'away_goals']])
            consolidated_results.append(result_xgb[['home_goals', 'away_goals']])

    # Calcular a média final dos resultados acumulados
    all_results = pd.concat(consolidated_results, axis=1)
    avg_home_goals = all_results.iloc[:, ::2].mean(axis=1)  # Média das colunas de home_goals
    avg_away_goals = all_results.iloc[:, 1::2].mean(axis=1)  # Média das colunas de away_goals

    # Criar DataFrame consolidado
    final_result = result_rf[['date', 'home_team_name', 'away_team_name']].copy()
    final_result['avg_home_goals'] = avg_home_goals
    final_result['avg_away_goals'] = avg_away_goals

    # Arredondando os gols para inteiros
    final_result['avg_home_goals_rounded'] = final_result['avg_home_goals'].round().astype(int)
    final_result['avg_away_goals_rounded'] = final_result['avg_away_goals'].round().astype(int)

    # Arredondando os gols para inteiros
    final_result['avg_home_goals_trunc'] = np.trunc(final_result['avg_home_goals']).astype(int)
    final_result['avg_away_goals_trunc'] = np.trunc(final_result['avg_away_goals']).astype(int)

    final_result['score_format_rounded'] = final_result.apply(lambda row: f"{row['avg_home_goals_rounded']}-{row['avg_away_goals_rounded']}", axis=1)
    final_result['score_format_truc'] = final_result.apply(lambda row: f"{row['avg_home_goals_trunc']}-{row['avg_away_goals_trunc']}", axis=1)

    final_result.drop(['avg_home_goals_rounded', 'avg_away_goals_rounded', 'avg_home_goals_trunc', 'avg_away_goals_trunc'], axis=1, inplace=True)


    # Exibir resultado consolidado
    display_html(
        f'''
        <div style="display: flex; justify-content: center;">
            <h3>League: {league}</h3>
        </div>
        ''',
        raw=True
    )
    display_html(
        final_result.to_html(index=False),
        raw=True
    )


League: EPL

date,home_team_name,away_team_name,avg_home_goals,avg_away_goals,score_format_rounded,score_format_truc
2025-05-10 14:00:00,Fulham,Everton,1.262959,0.831434,1-1,1-0
2025-05-10 14:00:00,Ipswich,Brentford,1.038313,3.319260,1-3,1-3
2025-05-10 14:00:00,Southampton,Manchester City,1.070531,2.672042,1-3,1-2
2025-05-10 14:00:00,Wolverhampton Wanderers,Brighton,1.588749,1.476705,2-1,1-1
2025-05-10 16:30:00,Bournemouth,Aston Villa,2.057155,0.990732,2-1,2-0
2025-05-11 11:00:00,Newcastle United,Chelsea,2.091014,1.495315,2-1,2-1
2025-05-11 13:15:00,Manchester United,West Ham,1.597082,1.140322,2-1,1-1
2025-05-11 13:15:00,Nottingham Forest,Leicester,1.954174,1.090443,2-1,1-1
2025-05-11 13:15:00,Tottenham,Crystal Palace,2.026127,1.519365,2-2,2-1
2025-05-11 15:30:00,Liverpool,Arsenal,1.644824,1.549016,2-2,1-1


League: La Liga

date,home_team_name,away_team_name,avg_home_goals,avg_away_goals,score_format_rounded,score_format_truc
2025-05-09 19:00:00,Las Palmas,Rayo Vallecano,1.798971,2.051046,2-2,1-2
2025-05-10 12:00:00,Valencia,Getafe,1.181126,0.636216,1-1,1-0
2025-05-10 14:15:00,Celta Vigo,Sevilla,1.752388,0.936786,2-1,1-0
2025-05-10 16:30:00,Girona,Villarreal,1.541194,1.352441,2-1,1-1
2025-05-10 16:30:00,Mallorca,Real Valladolid,2.678046,1.445690,3-1,2-1
2025-05-10 19:00:00,Atletico Madrid,Real Sociedad,1.965305,0.725616,2-1,1-0
2025-05-11 12:00:00,Leganes,Espanyol,1.033703,1.004235,1-1,1-1
2025-05-11 14:15:00,Barcelona,Real Madrid,1.950448,1.547979,2-2,1-1
2025-05-11 16:30:00,Athletic Club,Alaves,1.503873,0.677501,2-1,1-0
2025-05-11 19:00:00,Real Betis,Osasuna,2.035489,0.938711,2-1,2-0


League: Bundesliga

date,home_team_name,away_team_name,avg_home_goals,avg_away_goals,score_format_rounded,score_format_truc
2025-05-09 18:30:00,Wolfsburg,Hoffenheim,2.522506,2.185016,3-2,2-2
2025-05-10 13:30:00,Werder Bremen,RasenBallsport Leipzig,2.472597,1.633863,2-2,2-1
2025-05-10 13:30:00,Union Berlin,FC Heidenheim,1.798765,1.057119,2-1,1-1
2025-05-10 13:30:00,Bochum,Mainz 05,2.141916,2.395874,2-2,2-2
2025-05-10 13:30:00,Holstein Kiel,Freiburg,1.768821,2.106028,2-2,1-2
2025-05-10 16:30:00,Bayern Munich,Borussia M.Gladbach,2.821840,1.877050,3-2,2-1
2025-05-11 13:30:00,Bayer Leverkusen,Borussia Dortmund,1.798828,1.449241,2-1,1-1
2025-05-11 15:30:00,Eintracht Frankfurt,St. Pauli,2.125458,1.040267,2-1,2-1
2025-05-11 17:30:00,VfB Stuttgart,Augsburg,2.805946,0.904562,3-1,2-0


League: Serie A

date,home_team_name,away_team_name,avg_home_goals,avg_away_goals,score_format_rounded,score_format_truc
2025-05-09 18:45:00,AC Milan,Bologna,1.784930,1.105627,2-1,1-1
2025-05-10 13:00:00,Como,Cagliari,1.541466,0.992796,2-1,1-0
2025-05-10 16:00:00,Lazio,Juventus,1.057082,1.812020,1-2,1-1
2025-05-10 18:45:00,Empoli,Parma Calcio 1913,0.773388,0.936543,1-1,0-0
2025-05-11 10:30:00,Udinese,Monza,2.282170,0.569430,2-1,2-0
2025-05-11 13:00:00,Verona,Lecce,1.188188,1.021395,1-1,1-1
2025-05-11 16:00:00,Torino,Inter,1.204035,2.361202,1-2,1-2
2025-05-11 18:45:00,Napoli,Genoa,2.195541,0.522988,2-1,2-0
2025-05-12 16:30:00,Venezia,Fiorentina,1.151182,1.815303,1-2,1-1
2025-05-12 18:45:00,Atalanta,Roma,1.195236,1.151333,1-1,1-1


League: Ligue 1

date,home_team_name,away_team_name,avg_home_goals,avg_away_goals,score_format_rounded,score_format_truc
2025-05-10 19:00:00,Reims,Saint-Etienne,1.768790,0.864941,2-1,1-0
2025-05-10 19:00:00,Auxerre,Nantes,1.512301,1.021596,2-1,1-1
2025-05-10 19:00:00,Angers,Strasbourg,1.270782,1.378110,1-1,1-1
2025-05-10 19:00:00,Le Havre,Marseille,0.962835,2.330216,1-2,0-2
2025-05-10 19:00:00,Rennes,Nice,1.579781,1.444502,2-1,1-1
2025-05-10 19:00:00,Montpellier,Paris Saint Germain,1.032513,4.168626,1-4,1-4
2025-05-10 19:00:00,Toulouse,Lens,1.395733,1.224135,1-1,1-1
2025-05-10 19:00:00,Brest,Lille,1.248505,1.274174,1-1,1-1
2025-05-10 19:00:00,Monaco,Lyon,2.442061,1.433861,2-1,2-1
